In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys


# helper functions
sys.path.insert(0, "../src/lib")

import dataset as dataset_funcs
import cleaning as cleaning_funcs

pd.set_option('display.float_format', lambda x: '%.3f' % x)

**Approach and feature extraction**

The first approach is very naïve, because sometimes really simple models give good results and are easy and cheap to build (also, you can give some kind of early information to decision-makers while you are still working on a more sophisticated model).

Using **just sales data**, transform the dataset into a new dataset where each pair (X_i, y_i) is the following:

X_i = 3 (or 4, or 5) prices for 3 (or 4, or 5) sequential days (for a single product)
y_i = price for the next day

So we learn a very simple model that estimates the next price (for a single product)

> note that we have two subapproaches: in the first one we train on each product type separately and in the second one we train on all products togethers. This is meant to investigate whether we can increase performance when using data from all products.

**Assumptions**

- the model is stationary, i.e. the target function does not change over time, in other words, the way the previous prices affect the next one does not change depending upon what month of the year we're in.

**Shortcomings**

- we do not use data from competitors

**Evaluation**

- squared error, since we're dealing with continuous data

In [4]:
sales_df = pd.read_csv('../data/raw/sales.csv')
sales_df.DATE_ORDER = sales_df.DATE_ORDER.astype("datetime64")
sales_df["UNIT_PRICE"] = sales_df["REVENUE"] / sales_df["QTY_ORDER"]
sales_df.head()

,PROD_ID,DATE_ORDER,QTY_ORDER,REVENUE,UNIT_PRICE
0,P6,2015-08-02,1.000,1808.990,1808.990
1,P6,2015-08-17,1.000,1674.000,1674.000
2,P6,2015-08-17,1.000,1673.950,1673.950
3,P6,2015-08-11,1.000,1674.000,1674.000
4,P6,2015-08-17,1.000,1674.000,1674.000


right now the index is just numbers, but pandas allows us to inform that each index refers to a special Period in time (in this case, a day)

> note that using a PeriodIndex is different from just using a DatetimeIndex because then pandas would just think you want to index the data by a particular point in time, rather than by the whole day, as is the case here.

In [5]:
# periods = list(map(lambda dt: pd.Period(dt),sales_df["DATE_ORDER"]))
# idx = pd.PeriodIndex(periods)
# sales_df=sales_df.set_index(idx).drop(["DATE_ORDER"],axis=1)

as we mentioned on the **EXPLORATORY DATA ANALYSIS**, we will remove some bad data from our dataset to avoid propagating these errors to the model (using helper functions).

In [6]:
sales_df = cleaning_funcs.clean_sales_dataframe(sales_df);sales_df

,PROD_ID,DATE_ORDER,QTY_ORDER,REVENUE,UNIT_PRICE
0,P6,2015-08-02,1.000,1808.990,1808.990
1,P6,2015-08-17,1.000,1674.000,1674.000
2,P6,2015-08-17,1.000,1673.950,1673.950
3,P6,2015-08-11,1.000,1674.000,1674.000
4,P6,2015-08-17,1.000,1674.000,1674.000
5,P6,2015-07-30,1.000,1697.340,1697.340
6,P6,2015-08-06,1.000,1676.990,1676.990
7,P6,2015-08-16,1.000,1674.000,1674.000
8,P6,2015-07-25,1.000,1608.670,1608.670
9,P6,2015-07-16,1.000,1697.680,1697.680


Here we group by the product ID and the date the price was sampled.  

In [7]:
grouped_1 = sales_df.groupby([sales_df.PROD_ID,sales_df.DATE_ORDER],as_index=False).agg({
    "QTY_ORDER":np.sum,
    "REVENUE": np.sum,
    "UNIT_PRICE": np.mean
})
grouped_1.sample(10)


# len(sales_df[sales_df.DATE_ORDER < '2015-10-14 00:00:00'])
# len(sales_df[])


,PROD_ID,DATE_ORDER,REVENUE,QTY_ORDER,UNIT_PRICE
840,P4,2015-05-20,112335.660,232.000,483.783
1135,P5,2015-09-09,161252.050,205.000,786.597
1706,P7,2015-09-09,556959.090,800.000,696.274
744,P3,2015-08-30,2623.620,2.000,1311.810
2083,P9,2015-07-28,11400.670,26.000,438.445
45,P1,2015-03-21,17094.050,12.000,1424.504
945,P4,2015-09-02,24324.740,46.000,530.476
950,P4,2015-09-07,132568.890,301.000,440.628
474,P2,2015-08-11,76816.590,113.000,678.882
1465,P7,2015-01-09,184700.870,230.000,803.387


> note that the first approach treats each product individually

In [8]:
g1 = sales_df.groupby([sales_df.PROD_ID])

In [9]:
g1 = sales_df.groupby([sales_df.PROD_ID])
(p1,p2,p3,p4,p5,p6,p7,p8,p9) = [g1.get_group(prod_id) for prod_id in g1.groups.keys()]

dataset7 = p7.groupby(p7.DATE_ORDER).agg({
    "QTY_ORDER":np.sum,
    "UNIT_PRICE": np.mean
})
dataset7.sample(5)

,QTY_ORDER,UNIT_PRICE
DATE_ORDER,,
2015-03-21,7.000,594.721
2015-07-20,10.000,443.460
2015-05-12,43.000,493.399
2015-07-03,9.000,487.889
2015-09-01,758.000,374.886


## now that we have selected data only for P7, let's train a simple model on it

> remember, we'll use UNIT_PRICE and QTY_ORDER for the N=3 previous days to try to model what the next (4th) day. 

Note that we have **aggregated**, i.e. combined all rows for a given day (in the original dataset, there were multiple entries for the same product *and* the same day so we aggregated those).

In [10]:
dataset7.sort_index(inplace=True)
dataset7.reset_index(inplace=True)
dataset7[dataset7.DATE_ORDER.dt.month==5]

,DATE_ORDER,QTY_ORDER,UNIT_PRICE
43,2015-05-01,6.000,535.167
44,2015-05-02,9.000,538.000
45,2015-05-03,10.000,550.200
46,2015-05-04,30.000,505.724
47,2015-05-05,37.000,511.293
48,2015-05-06,22.000,512.781
49,2015-05-07,63.000,496.953
50,2015-05-08,21.000,515.779
51,2015-05-09,34.000,504.380
52,2015-05-10,38.000,505.829


In [15]:
for row in dataset7.values:
    print(row[2])

539.1
561.5288888888889
594.7214285714286
599.0
581.8857142857142
554.4726666666668
543.7076923076921
599.0
599.0
599.0
599.0
561.202
599.0
534.5975
444.29173913043485
491.2421874999999
492.9515151515151
496.505
547.0
569.05
599.0
599.0
581.03
499.905625
498.688
484.4665909090909
488.5846774193548
480.0868852459016
498.7903278688524
498.1692857142857
484.5370833333333
494.1088235294118
503.9504347826086
491.2799999999999
504.67699999999996
501.2947368421052
496.20799999999997
499.50499999999994
503.8081249999999
502.474
514.4774999999998
538.0
516.3923076923077
535.1666666666666
538.0
550.2
505.7241379310345
511.2929729729731
512.7809090909091
496.95327868852456
515.7785714285712
504.38000000000005
505.82945945945943
538.15
493.39883720930237
474.84203883495144
461.51652996845496
458.72052023121483
461.00807142857224
459.91555555555584
472.8820930232557
501.8052631578947
496.7053333333331
498.7188888888889
505.2142857142857
489.33500000000004
466.0414814814816
469.4519999999999
435.767

In [ ]:
import dataset as dataset_funcs

dataset7[dataset7.iloc[:,0] == pd.tslib.Timestamp('2015-06-30 00:00:00')].index.tolist()[0]

# type(dataset7.iloc[0])
# type(dataset7.iloc[:,0])

In [16]:
arr = np.array([1,2,3,4])

In [45]:
lst = [[2,3,4],[4,5,6]]

In [46]:
np.array(lst)

array([[2, 3, 4],
       [4, 5, 6]])

In [47]:
lst1 = []
lst1.append(lst)
lst1

[[[2, 3, 4], [4, 5, 6]]]

In [51]:
arr.tolist()

[1, 2, 3, 4]